# Neo4j Graph Database with Cypher Queries using Python

This project demonstrates how to build and query a graph database using Neo4j and Cypher with Python integration. It covers persons, movies, actors, directors, and genres—plus exporting query results to Excel.

In [77]:
!pip install neo4j pandas matplotlib

## Step 1: Connect to the Neo4j Database
Using the Neo4j Bolt driver to connect via Python.

In [79]:
from neo4j import GraphDatabase

# Neo4j credentials
uri = "bolt://localhost:7687"
username = "neo4j"
password = "qwertyuiop"  # change if you used something else

# Connect to DB
driver = GraphDatabase.driver(uri, auth=(username, password))

# Test query
def test_connection(tx):
    return tx.run("RETURN 'Neo4j Connected ✅' AS message").single()["message"]

with driver.session() as session:
    msg = session.execute_read(test_connection)
    print(msg)


Neo4j Connected ✅


## Step 2: Insert Sample Person Nodes & Relationships
Creating a basic "Friends With" graph for demonstration.

In [134]:
def create_data(tx):
    tx.run("MATCH (n) DETACH DELETE n")  # Clear all nodes + relationships
    tx.run("""
        CREATE (a:Person {name: 'Gaurav', age: 25})
        CREATE (b:Person {name: 'Akash', age: 30})
        CREATE (c:Person {name: 'Charlie', age: 28})
        CREATE (a)-[:FRIENDS_WITH]->(b)
        CREATE (b)-[:FRIENDS_WITH]->(c)
    """)

with driver.session() as session:
    session.execute_write(create_data)

print("Person graph created")

Person graph created


###  View Person–Friend Relationships
Cypher query to display all FRIENDS_WITH relationships.

In [119]:
def fetch_data(tx):
    result = tx.run("""
        MATCH (p:Person)-[r:FRIENDS_WITH]->(f:Person)
        RETURN p.name AS Person, f.name AS Friend
    """)
    return result.data()

with driver.session() as session:
    data = session.read_transaction(fetch_data)
import pandas as pd
df = pd.DataFrame(data)
print(df)


   Person   Friend
0  Gaurav    Akash
1   Akash  Charlie


C:\Users\PC\AppData\Local\Temp\ipykernel_24692\3904941150.py:9: DeprecationWarning: read_transaction has been renamed to execute_read
  data = session.read_transaction(fetch_data)


##  Step 3: Insert Movie Nodes Dynamically
Adding movies with title, year, and rating using Python lists.

In [138]:
# Inserting Movies
def insert_movie_data(tx):
    movies = [
        {"title": "Dilwale", "year": 2015, "rating": 6.0},
        {"title": "3 Idiots", "year": 2009, "rating": 8.4},
        {"title": "Chak De India", "year": 2007, "rating": 8.2},
        {"title": "PK", "year": 2014, "rating": 8.1},
        {"title": "Dangal", "year": 2016, "rating": 8.3},
        {"title": "Don", "year": 2006, "rating": 7.2},
        {"title": "Swades", "year": 2004, "rating": 8.2},
        {"title": "Taare Zameen Par", "year": 2007, "rating": 8.4},
        {"title": "My Name Is Khan", "year": 2010, "rating": 8.0},
        {"title": "Lagaan", "year": 2001, "rating": 8.1}
    ]
    for movie in movies:
        tx.run("""
            MERGE (:Movie {title: $title, year: $year, rating: $rating})
        """, title=movie["title"], year=movie["year"], rating=movie["rating"])

with driver.session() as session:
    session.execute_write(insert_movie_data)

print(" Movies inserted successfully.")

 Movies inserted successfully.


## Step 4: Link Actors & Directors to Movies
Creating ACTED_IN and DIRECTED relationships dynamically.

In [140]:
def insert_actors_directors(tx):
    data = [
        {"movie": "Dilwale", "actor": "Shah Rukh Khan", "actor_age": 58, "director": "Rohit Shetty", "director_age": 50},
        {"movie": "3 Idiots", "actor": "Aamir Khan", "actor_age": 59, "director": "Rajkumar Hirani", "director_age": 61},
        {"movie": "Chak De India", "actor": "Shah Rukh Khan", "actor_age": 58, "director": "Shimit Amin", "director_age": 50},
        {"movie": "PK", "actor": "Aamir Khan", "actor_age": 59, "director": "Rajkumar Hirani", "director_age": 61},
        {"movie": "Dangal", "actor": "Aamir Khan", "actor_age": 59, "director": "Nitesh Tiwari", "director_age": 49},
        {"movie": "Don", "actor": "Shah Rukh Khan", "actor_age": 58, "director": "Farhan Akhtar", "director_age": 50},
        {"movie": "Swades", "actor": "Shah Rukh Khan", "actor_age": 58, "director": "Ashutosh Gowariker", "director_age": 60},
        {"movie": "Taare Zameen Par", "actor": "Darsheel Safary", "actor_age": 26, "director": "Aamir Khan", "director_age": 59},
        {"movie": "My Name Is Khan", "actor": "Shah Rukh Khan", "actor_age": 58, "director": "Karan Johar", "director_age": 51},
        {"movie": "Lagaan", "actor": "Aamir Khan", "actor_age": 59, "director": "Ashutosh Gowariker", "director_age": 60}
    ]

    for entry in data:
        tx.run("""
            MATCH (m:Movie {title: $movie})
            MERGE (a:Actor {name: $actor}) SET a.age = $actor_age
            MERGE (d:Director {name: $director}) SET d.age = $director_age
            MERGE (a)-[:ACTED_IN]->(m)
            MERGE (d)-[:DIRECTED]->(m)
        """, **entry)

with driver.session() as session:
    session.execute_write(insert_actors_directors)

print(" Actors & Directors linked to movies successfully.")

 Actors & Directors linked to movies successfully.


###  Query: Fetch Movies with Actor & Director Info
Matches all (Actor)-[:ACTED_IN]->(Movie)<-[:DIRECTED]-(Director) links.

In [142]:
# This will be empty unless actor+director nodes/relations exist
def fetch_movie_info(tx):
    result = tx.run("""
        MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(d:Director)
        RETURN DISTINCT m.title AS Movie, a.name AS Actor, d.name AS Director
    """)
    return result.data()

with driver.session() as session:
    movie_info = session.execute_read(fetch_movie_info)

df_info = pd.DataFrame(movie_info)
print(df_info)  # may be empty if no actors/directors created

              Movie            Actor            Director
0            Dangal       Aamir Khan       Nitesh Tiwari
1               Don   Shah Rukh Khan       Farhan Akhtar
2            Swades   Shah Rukh Khan  Ashutosh Gowariker
3            Lagaan       Aamir Khan  Ashutosh Gowariker
4  Taare Zameen Par  Darsheel Safary          Aamir Khan
5   My Name Is Khan   Shah Rukh Khan         Karan Johar
6           Dilwale   Shah Rukh Khan        Rohit Shetty
7          3 Idiots       Aamir Khan     Rajkumar Hirani
8                PK       Aamir Khan     Rajkumar Hirani
9     Chak De India   Shah Rukh Khan         Shimit Amin


### Query: Fetch Movies by Specific Actor
Searches all movies connected to the given actor’s name.


In [145]:
#  Movies by Actor (this will also be empty unless actors are created)
def movies_by_actor(tx, actor_name):
    result = tx.run("""
        MATCH (a:Actor {name: $name})-[:ACTED_IN]->(m:Movie)
        RETURN DISTINCT m.title AS Movie, m.year AS Year
    """, name=actor_name)
    return result.data()

with driver.session() as session:
    actor_movies = session.execute_read(movies_by_actor, "Shah Rukh Khan")

df_actor = pd.DataFrame(actor_movies)
print(df_actor)

             Movie  Year
0          Dilwale  2015
1    Chak De India  2007
2              Don  2006
3           Swades  2004
4  My Name Is Khan  2010


## Step 5: Add Genres & Connect to Movies
Linking each movie to a genre using BELONGS_TO relationship.


In [148]:
def insert_genres(tx):
    genre_map = {
        "Dilwale": "Action",
        "3 Idiots": "Drama",
        "Chak De India": "Sports",
        "PK": "Comedy",
        "Dangal": "Biopic",
        "Don": "Action",
        "Swades": "Drama",
        "Taare Zameen Par": "Emotional",
        "My Name Is Khan": "Drama",
        "Lagaan": "Sports"
    }

    for movie, genre in genre_map.items():
        tx.run("""
            MERGE (g:Genre {name: $genre})
            WITH g
            MATCH (m:Movie {title: $movie})
            MERGE (m)-[:BELONGS_TO]->(g)
        """, movie=movie, genre=genre)

with driver.session() as session:
    session.execute_write(insert_genres)

print(" Genres added and linked to movies successfully.")

 Genres added and linked to movies successfully.


### Query: View All Movies with Their Genres
Sorted by genre, for better understanding of categories.

In [151]:
def fetch_movies_with_genres(tx):
    result = tx.run("""
        MATCH (m:Movie)-[:BELONGS_TO]->(g:Genre)
        RETURN m.title AS Movie, g.name AS Genre
        ORDER BY Genre, Movie
    """)
    return result.data()

with driver.session() as session:
    genre_results = session.execute_read(fetch_movies_with_genres)

import pandas as pd
df_genres = pd.DataFrame(genre_results)
print(df_genres)

              Movie      Genre
0           Dilwale     Action
1               Don     Action
2            Dangal     Biopic
3                PK     Comedy
4          3 Idiots      Drama
5   My Name Is Khan      Drama
6            Swades      Drama
7  Taare Zameen Par  Emotional
8     Chak De India     Sports
9            Lagaan     Sports


## Step 6: Export All Query Results to Excel
All results (friends, actors, genres) exported to a multi-sheet Excel file.

In [154]:
with pd.ExcelWriter("final_graph_project.xlsx") as writer:
    df_person.to_excel(writer, sheet_name="Friends Network", index=False)
    df_info.to_excel(writer, sheet_name="Actor-Director", index=False)
    df_actor.to_excel(writer, sheet_name="Movies by Actor", index=False)
    df_genres.to_excel(writer, sheet_name="Movies by Genre", index=False)

print(" All data saved into final_graph_project.xlsx")

 All data saved into final_graph_project.xlsx
